In [1]:
import numpy as np
import pandas as pd
from scipy.fft import fft, ifft
from scipy.interpolate import interp1d
from scipy.stats import entropy, iqr, kurtosis, mode, skew
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import (Activation, BatchNormalization, Bidirectional,
                                     Conv1D, Conv2D, Dense, Dropout,
                                     Flatten, GlobalAveragePooling1D, LSTM,
                                     MaxPool1D, MaxPooling2D, Reshape,
                                     TimeDistributed)
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import os
import pickle
from pandas import read_csv, unique
from scipy.signal import argrelextrema, butter, find_peaks, lfilter, lfilter_zi
from tensorflow import keras
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_accuracy
from kerastuner import RandomSearch, Hyperband
import kerastuner


2023-11-07 11:50:24.165704: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-07 11:50:24.465540: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-07 11:50:24.465589: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-07 11:50:25.898244: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [8]:
# load tflite model
interpreter = tf.lite.Interpreter(model_path="APP_READY_cnn_lstm_physical.tflite")
interpreter.allocate_tensors()
# load tf model
model = keras.models.load_model('Trained_Task1_2.keras')


In [17]:
subactiv = ['coughing', 'eating', 'hyperventilating', 'laughing', 'normal', 'singing', 'talking']
phys = ['ascending stairs', 'descending stairs', 'lying down back',
        'lying down on left', 'lying down on stomach', 'lying down right',
        'miscellaneous movements', 'normal walking', 'running',
        'shuffle walking', 'sitting', 'standing']

In [6]:
X_respeck = pickle.load(open("X_respeck_all_phys", "rb"))
y_respeck = pickle.load(open("y_respeck_all_phys", "rb"))

In [37]:
X_train, X_temp, y_train, y_temp = train_test_split(X_respeck, y_respeck, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [30]:
len(X_respeck[0])

50

In [21]:
# test model
pred = model.predict(X_respeck)
pred = np.argmax(pred, axis=1)
pred = [phys[i] for i in pred]

   2/1792 [..............................] - ETA: 1:32

2023-11-07 12:04:57.069408: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 114670000 exceeds 10% of free system memory.


1792/1792 [==============================] - 80s 45ms/step


In [31]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 192)           155904    
                                                                 
 lstm_1 (LSTM)               (None, 50, 224)           373632    
                                                                 
 conv1d (Conv1D)             (None, 24, 96)            86112     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 8, 96)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 6, 96)             27744     
                                                                 
 global_average_pooling1d (G  (None, 96)               0         
 lobalAveragePooling1D)                                 

In [22]:
print(classification_report(y_respeck, pred))
print(confusion_matrix(y_respeck, pred))


                         precision    recall  f1-score   support

       ascending stairs       0.98      0.99      0.98      1299
      descending stairs       0.99      0.98      0.99      1298
        lying down back       1.00      1.00      1.00      7816
     lying down on left       1.00      0.99      0.99      7817
  lying down on stomach       1.00      1.00      1.00      7820
       lying down right       0.99      1.00      1.00      7818
miscellaneous movements       0.99      0.99      0.99      1306
         normal walking       1.00      1.00      1.00      1302
                running       1.00      1.00      1.00      1306
        shuffle walking       1.00      0.99      1.00      1306
                sitting       0.94      0.97      0.96      9120
               standing       0.97      0.94      0.96      9127

               accuracy                           0.98     57335
              macro avg       0.99      0.99      0.99     57335
           weighted avg

In [24]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Configure post-training quantization options
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Disable experimental lowering of tensor list ops and enable Select TF ops
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False

# Convert the model to TFLite with the updated settings
tflite_model = converter.convert()

# Save the TFLite model to a file
with open('your_model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmp42v1pxie/assets


INFO:tensorflow:Assets written to: /tmp/tmp42v1pxie/assets
2023-11-07 12:08:08.936043: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-11-07 12:08:08.936090: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-11-07 12:08:08.936256: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp42v1pxie
2023-11-07 12:08:08.942303: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-11-07 12:08:08.942361: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp42v1pxie
2023-11-07 12:08:08.979337: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-11-07 12:08:09.084345: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: /tmp/tmp42v1pxie
2023-11-07 12:08:09.147570: I tensorflow/cc/saved_model/loader.cc:305] SavedModel

In [38]:


# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='your_model.tflite')
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Perform inference and evaluate the model
predictions = []
for data in X_test:
    # Set the input data
    interpreter.set_tensor(input_details[0]['index'], np.array([data.astype(np.float32)]))

    # Run the inference
    interpreter.invoke()

    # Get the output
    output_data = interpreter.get_tensor(output_details[0]['index'])
    print(phys[output_data.argmax()])
    predictions.append(phys[output_data.argmax()])


lying down on left
lying down on stomach
lying down back
lying down back
lying down on stomach
miscellaneous movements
lying down on stomach
normal walking
miscellaneous movements
sitting
sitting
running
lying down on left
lying down on stomach
lying down on stomach
lying down back
sitting
sitting
lying down on left
running
descending stairs
lying down on stomach
lying down on left
lying down on stomach
sitting
lying down on left
lying down right
miscellaneous movements
lying down on stomach
lying down back
sitting
standing
lying down right
sitting
lying down back
lying down back
lying down right
standing
sitting
lying down on left
lying down on stomach
lying down on stomach
standing
lying down right
running
lying down on stomach
miscellaneous movements
lying down on left
lying down on stomach
sitting
lying down right
normal walking
sitting
sitting
lying down on stomach
lying down on stomach
shuffle walking
sitting
standing
lying down on left
sitting
lying down back
lying down on left


In [41]:
from sklearn.metrics import accuracy_score
# Calculate accuracy
accuracy = accuracy_score(y_test, predictions)
accuracy
confusion_matrix(y_test, predictions)

array([[132,   2,   0,   0,   0,   0,   1,   0,   0,   0,   0,   0],
       [  5, 142,   0,   0,   0,   0,   1,   0,   0,   1,   0,   0],
       [  0,   0, 777,   0,   0,   2,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0, 756,   2,   3,   0,   0,   0,   0,   3,   0],
       [  0,   0,   0,   2, 783,   1,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0, 792,   0,   0,   0,   0,   0,   1],
       [  0,   0,   0,   1,   0,   0, 143,   0,   1,   0,   1,   0],
       [  3,   0,   0,   0,   0,   0,   0, 121,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0, 117,   0,   0,   0],
       [  1,   0,   0,   0,   0,   0,   1,   0,   0, 136,   0,   0],
       [  1,   1,   0,   0,   0,   1,   0,   0,   0,   1, 868,  38],
       [  1,   0,   2,   0,   0,   1,   0,   0,   0,   0, 139, 749]])